# 1. Data Generator
- Raw Data를 읽어옴
- 여기서 만들어진 데이터는 모델의 입력으로 들어감

In [7]:
import os
import numpy as np
import librosa
from tensorflow.keras.utils import Sequence

In [220]:
class RawForVAEGenerator(Sequence):
    def __init__(self, source, wav_dir, files, sourNum='s1', batch_size=10, shuffle=True):
        self.source = source
        self.wav_dir = wav_dir
        self.files = files
        self.sourNum = sourNum
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()
        
        self.sample_rate = 8000
    
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.source))
        
        if self.shuffle:
            np.random.shuffle(self.indexes)
    
    def __audioread__(self, path, offset=0.0, duration=None, sample_rate=16000):
        signal = librosa.load(path, sr=self.sample_rate, mono=False, offset=offset, duration=duration)

        return signal[0]
    
    def __padding__(self, data):
        n_batch = len(data)
        max_len = max([d.shape[0] for d in data])
        extrapadding = int(np.ceil(max_len / self.sample_rate) * self.sample_rate)
        pad = np.zeros((n_batch, extrapadding))
        
        for i in range(n_batch):
            pad[i, :data[i].shape[0]] = data[i]
        
        return np.expand_dims(pad, -1)
        
    def __data_generation__(self, source_list):
        wav_list = []
        for name in source_list:
            name = name.strip('\n')
            
            s_wav_name = self.wav_dir + self.files + '/' + self.sourNum + '/' + name
            
            # ------- AUDIO READ -------
            s_wav = (self.__audioread__(s_wav_name,  offset=0.0, duration=None, sample_rate=self.sample_rate))
            # --------------------------
            
            # ------- PADDING -------
#             pad_len = max(len(samples1),len(samples2))
#             pad_s1 = np.concatenate([s1_wav, np.zeros([pad_len - len(s1_wav)])])
            
#             extrapadding = ceil(len(pad_s1) / sample_rate) * sample_rate - len(pad_s1)
#             pad_s1 = np.concatenate([pad_s1, np.zeros([extrapadding - len(pad_s1)])])
#             pad_s2 = np.concatenate([s2_wav, np.zeros([extrapadding - len(s2_wav)])])
            # -----------------------
            
            wav_list.append(s_wav)
        
        return wav_list, wav_list
            
    
    def __len__(self):
        return int(np.floor(len(self.source) / self.batch_size))
    
    def __getitem__(self, index):
        indexes = self.indexes[index * self.batch_size : (index + 1) * self.batch_size]
        source_list = [self.source[k] for k in indexes]
        
        if self.files is not 'tt':
            sour, labels = self.__data_generation__(source_list)
            
            # Get Lengths(K value of each batch)
            lengths = np.array([m.shape[0] for m in sour])
            exp = np.expand_dims(lengths, 1)
            exp = np.expand_dims(exp, -1) # [Batch, 1, 1] (length)
            
            # Padding
            sour_pad = self.__padding__(sour) # [Batch, Time_step, Dimension(=1)]
            label_pad = self.__padding__(labels) # [Batch, Time_step, Dimension(=1)]
            
            return sour_pad, np.concatenate([label_pad, exp], axis=1)
        else:
            sour, labels = self.__data_generation__(source_list)
            
            # Get Lengths(K value of each batch)
            lengths = np.array([m.shape[0] for m in sour])
            exp = np.expand_dims(lengths, 1)
            exp = np.expand_dims(exp, -1) # [Batch, 1, 1] (length)
            
            # Padding
            sour_pad = self.__padding__(sour) # [Batch, Time_step, Dimension(=1)]
            
            return sour_pad, exp

## Data를 어떻게 읽는지에 대한 부분

In [221]:
WAV_DIR = './mycode/wsj0_2mix/use_this/'
LIST_DIR = './mycode/wsj0_2mix/use_this/lists/'

In [222]:
# Directory List file create

wav_dir = WAV_DIR
output_lst = LIST_DIR

for folder in ['tr', 'cv', 'tt']:
    wav_files = os.listdir(wav_dir + folder + '/mix')
    output_lst_files = output_lst + folder + '_wav.lst'
    with open(output_lst_files, 'w') as f:
        for file in wav_files:
            f.write(file + "\n")

print("Generate wav file to .lst done!")

Generate wav file to .lst done!


In [324]:
batch_size = 2

train_dataset = 0
valid_dataset = 0
test_dataset = 0

name_list = []
for files in ['tr', 'cv', 'tt']:
    # --- Lead lst file ---""
    output_lst_files = LIST_DIR + files + '_wav.lst'
    fid = open(output_lst_files, 'r')
    lines = fid.readlines()
    fid.close()
    # ---------------------
    
    if files == 'tr':
        train_dataset = RawForVAEGenerator(lines, WAV_DIR, files, 's1', batch_size)
    elif files == 'cv':
        valid_dataset = RawForVAEGenerator(lines, WAV_DIR, files, 's1', batch_size)
    else:
        test_dataset = RawForVAEGenerator(lines, WAV_DIR, files, 's1', batch_size)
a, b = next(iter(train_dataset))
b.shape

(2, 80001, 1)

# 2. Building VQ-VAE model with Gumbel Softmax

In [385]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
from tensorflow.keras import backend as Kb
import numpy as np
import pandas as pd
from importlib import reload
import time

In [378]:
def mkdir_p(path):
    """ Creates a path recursively without throwing an error if it already exists
    :param path: path to create
    :return: None
    """
    if not os.path.exists(path):
        os.makedirs(path)

In [379]:
mkdir_p('./CKPT/') # model check point 폴더 만드는 코드
filepath = "./CKPT/CKP_ep_{epoch:d}__loss_{val_loss:.5f}_.h5"

In [380]:
initial_learning_rate = 0.001

# learning rate를 점점 줄이는 부분
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=20, decay_rate=0.96, staircase=True
)

# validation loss에 대해서 좋은 것만 저장됨
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min'
)

# early stop 하는 부분인데, validation loss에 대해서 제일 좋은 모델이 저장됨
early_stopping_cb = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', mode='min', verbose=1, patience=50, restore_best_weights=True
)

In [381]:
def vae_loss(q_y, latent_dim):
    def si_sdr_loss(y_true, y_pred):
        ori_length = tf.shape(y_true)[1]

        # Label & Length divide
        labels = tf.slice(y_true, [0, 0, 0], [-1, ori_length-1, -1]) # [batch_size, length_size, 1]
        lengths = tf.slice(y_true, [0, ori_length-1, 0], [-1, -1, 1]) # [batch_size, 1, 1]

        
        target = tf.reduce_sum([y_pred, labels]) * labels / tf.experimental.numpy.square(labels)
        noise = y_pred - target
        si_sdr = 10 * np.log10(pow_np_norm(target) / tf.experimental.numpy.square(noise))
        sdr = si_sdr * -0.5

        return sdr
    
    log_q_y = tf.log(q_y+1e-20)
    kl_loss = tf.reduce_sum(q_y*(log_q_y-tf.log(1.0/latent_dim)), axis=[1,2])
    
    return si_sdr_loss - kl_loss

In [392]:
class Vq_vae(keras.Model):
    def __init__(self, latent_dim):
        super(Vq_vae, self).__init__(name='vqvae')
        self.latent_dim = latent_dim
        self.encoder = keras.Sequential(
            [
                layers.InputLayer(input_shape=(None, 1)),
                layers.Conv1D(
                    filters=64, kernel_size=3, strides=2, activation='relu', padding='same'),
                layers.Conv1D(
                    filters=64, kernel_size=3, strides=2, activation='relu', padding='same'),
                # No activation
                layers.Dense(latent_dim * latent_dim),
                layers.Reshape((latent_dim, latent_dim)),
            ]
        )

        self.decoder = keras.Sequential(
            [
                layers.InputLayer(input_shape=(latent_dim, latent_dim,)),
                layers.Flatten(),
                layers.Dense(units=7*32, activation='relu'),
                layers.Reshape((7, 32)),
                layers.Conv1DTranspose(
                    filters=64, kernel_size=3, strides=2, padding='same', activation='relu'),
                layers.Conv1DTranspose(
                    filters=32, kernel_size=3, strides=2, padding='same', activation='relu'),
                # No activation
                layers.Conv1DTranspose(
                    filters=1, kernel_size=3, strides=1, padding='same'),
            ]
        )
    
    def sample_gumbel(shape, eps=1e-20): 
        """Sample from Gumbel(0, 1)"""
        U = tf.random_uniform(shape,minval=0,maxval=1)
        
        return -tf.log(-tf.log(U + eps) + eps)

    def gumbel_softmax_sample(logits, temperature): 
        """ Draw a sample from the Gumbel-Softmax distribution"""
        y = logits + sample_gumbel(tf.shape(logits))
        
        return tf.nn.softmax( y / temperature)

    def gumbel_softmax(logits, temperature, hard=False):
        y = gumbel_softmax_sample(logits, temperature)
        if hard:
            k = tf.shape(logits)[-1]
            #y_hard = tf.cast(tf.one_hot(tf.argmax(y,1),k), y.dtype)
            y_hard = tf.cast(tf.equal(y,tf.reduce_max(y,1,keep_dims=True)),y.dtype)
            y = tf.stop_gradient(y_hard - y) + y
        
        return y
    
    def call(self, inputs):
        encoder = self.encoder(inputs)
        z_latent = self.gumbel_softmax(tf.reshape(encoder, [-1, latent_dim]), 0.9)
        decoder = self.decoder(z)
        
        return encoder, decoder

In [390]:
# def compute_loss(model, x, latent_dim):
#     enc, latent = model.encode(x)
#     x_logit = model.decode(enc)
#     mse_loss = tf.reduce_sum(keras.metrics.mean_squared_error(x, x_logit), axis=[-1])
    
#     q_y = tf.nn.softmax(enc, axis=-1)
#     log_q_y = tf.log(q_y+1e-20)
#     kl_loss = tf.reduce_sum(q_y*(log_q_y-tf.log(1.0/latent_dim)), axis=[1,2])
#     elbo = mse_loss - kl_loss
#     loss=tf.reduce_mean(-elbo)
    
#     return loss

# def train_step(model, x, optimizer, latent_size):
#     with tf.GradientTape() as tape:
#         loss = compute_loss(model, x, latent_size)
#     gradients = tape.gradient(loss, model.trainable_variables)
#     optimizer.apply_gradients(zip(gradients, model.trainable_variables))

In [398]:
# epochs = 2
# latent_size = 512
# model = Vq_vae(latent_size)
# optimizer = tf.keras.optimizers.Adam(1e-4)

# for epoch in range(1, epochs + 1):
#     start_time = time.time()
#     for train_x in train_dataset:
#         train_step(model, train_x, optimizer, latent_size)
#     end_time = time.time()

#     loss = keras.metrics.Mean()
#     for valid_x in valid_dataset:
#         loss(compute_loss(model, valid_x))
#     elbo = -loss.result()
#     display.clear_output(wait=False)
#     print('Epoch: {}, Test set ELBO: {}, time elapse for current epoch: {}'
#         .format(epoch, elbo, end_time - start_time))
#     generate_and_save_images(model, epoch, test_sample)

In [397]:
latent_size = 512
epoch = 5
BATCH_SIZE = 2

strategy = tf.distribute.MirroredStrategy()
print('장치의 수: {}'.format(strategy.num_replicas_in_sync))

with strategy.scope():
    # 사용 안할 때는 load_model 주석 처리 하자
#     model = load_model('./CKPT/CKP_ep_29__loss_102.63367_.h5', custom_objects={'pit_loss': pit_with_outputsize(OUTPUT_SIZE)})
    model = Vq_vae(latent_size)
    adam = tf.optimizers.Adam(learning_rate=lr_schedule)
    model.compile(loss=keras.losses.mean_squared_error, optimizer=adam)
    tf.executing_eagerly()

history = model.fit(
    train_dataset,
    epochs=epoch,
    validation_data=valid_dataset,
    shuffle=True,
    callbacks=[checkpoint_cb, early_stopping_cb],
)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
장치의 수: 1
INFO:tensorflow:Error reported to Coordinator: OOM when allocating tensor with shape[2,20000,262144] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu [Op:BiasAdd]
Traceback (most recent call last):
  File "c:\users\moon\anaconda3\envs\study\lib\site-packages\tensorflow\python\training\coordinator.py", line 297, in stop_on_exception
    yield
  File "c:\users\moon\anaconda3\envs\study\lib\site-packages\tensorflow\python\distribute\mirrored_run.py", line 334, in run
    self.main_result = self.main_fn(*self.main_args, **self.main_kwargs)
  File "c:\users\moon\anaconda3\envs\study\lib\site-packages\tensorflow\python\autograph\impl\api.py", line 597, in wrapper
    return func(*args, **kwargs)
  File "c:\users\moon\anaconda3\envs\study\lib\site-packages\tensorflow\python\keras\engine\data_adapter.py", line 817, in <lambda>
    lambda x: model(x, trai

ResourceExhaustedError: OOM when allocating tensor with shape[2,20000,262144] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu [Op:BiasAdd]